<a href="https://colab.research.google.com/github/iliodipietro/AIML_Project/blob/master/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Install requirements**

In [0]:
!pip3 install 'torch==1.3.1'
!pip3 install 'torchvision==0.4.2'
!pip3 uninstall 'Pillow-SIMD'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'
!pip install --upgrade pillow

!pip uninstall 'tensorflow==2.0.0'
!pip install 'tensorflow'

**Import libraries**

In [0]:
import os
import cv2
import logging
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from   matplotlib.pyplot import subplots

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from torchvision.models import alexnet #serve? 

from tqdm import tqdm
import pandas as pd 

import re
import urllib
from urllib import request, error
from urllib.request import urlopen, HTTPError
from io import BytesIO
from PIL import Image
import random

from google.colab import drive

import sys, requests, shutil
from shutil import copyfile

from IPython.display import Image
from IPython.core.display import HTML 

#togliere nel caso non usassimo piu la classe landmark
from torchvision.datasets import VisionDataset
import os.path

**Loading the dataset from Google Drive**

In [0]:
drive.mount('/content/drive')

**Define Data Preprocessing**

In [0]:
# transformation for TRANSFER LEARNING, based on the ImageNet dataset
normalize = transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
train_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.RandomCrop(224),
                                      transforms.RandomRotation(20),
                                      transforms.ToTensor(),
                                      normalize,])

eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      normalize,])

**Plot data distribution and filter num.classes**

In [0]:
#Import csv files containing dataset data structure 
#each row of the file contains 'id,URL,Landmark_ID'. First row is the header of the file

data = pd.read_csv('/content/drive/My Drive/Progetto_AIML/dataset/train.csv')      #('/content/Project_AIML/train.csv')
test_data = pd.read_csv('/content/drive/My Drive/Progetto_AIML/dataset/test.csv')    #('/content/Project_AIML/test.csv')
data.head(5)

#DATA SAMPLES
#study distributions of data from selected filtered classes 1000 - 1299

#we define a list of labels from 1000 to 1299
landmark_list = [str(x) for x in list(range(1000,1300))]
#which images stored in data match the landmark ID from 1000 to 1299? check it with data.isin(landmark_list)
data_sample = data[data['landmark_id'].isin(landmark_list)]
# data_sample.landmark_id.value_counts()
print(len(data_sample))
# Check data distribution
%matplotlib inline


colors = np.array(['#4285f4','#34a853','#fbbc05','#ea4335'])
#Define the order in which to display the graph
order = ['1-5','5-10','10-50','50-100','100-200','200-500','>=500']
f, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,5))


def plot_distribution(data_f, data_k, axis):
    #data['landmark_id'].value_counts()
    x=data_f.landmark_id.value_counts().index
    y=pd.DataFrame(data_f.landmark_id.value_counts())

    #Create a variable to group the number of image sin each class
    y['Number of images'] = np.where(y['landmark_id']>=500,'>=500',y['landmark_id'])
    y['Number of images'] = np.where((y['landmark_id']>=200) & (y['landmark_id']<500),'200-500',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=100) & (y['landmark_id']<200),'100-200',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=50) & (y['landmark_id']<100),'50-100',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=10) & (y['landmark_id']<50),'10-50',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=5) & (y['landmark_id']<10),'5-10',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=0) & (y['landmark_id']<5),'1-5',y['Number of images'])

    y['Number of images'].value_counts().loc[order].plot(kind = 'bar',color = colors,width = 0.8, ax=axis, fontsize = 14)
    axis.set_xlabel('Number of images', fontsize = 18)
    axis.set_ylabel('Number of classes', fontsize = 18)
    axis.grid()
    axis.set_title(data_k, fontsize = 18)
    
plot_distribution(data, 'Total', ax1)
plot_distribution(data_sample, 'Filtered', ax2)

 **Display Photo**

In [0]:
def displayLandmarkImagesLarge(urls, category_name):
    img_style = "width: 200px; height:160px; margin: 0px; float: left; border: 1px solid black;"
    images_list = ''.join([f"<img style='{img_style}' src='{u}' />" for _, u in urls.head(12).iteritems()])
    display(HTML(images_list))


category = data['landmark_id'].value_counts().keys()[15]
urls = data[data['landmark_id'] == category]['url']
displayLandmarkImagesLarge(urls, "")

# Visualize 5 images for each of the first 4 landmarks, ordered by the number of occurences.
LANDMARK_NUMBER = 10
IMAGES_NUMBER = 15
landMarkIDs = pd.Series(data['landmark_id'].value_counts().keys())[0:LANDMARK_NUMBER]
for landMarkID in landMarkIDs:
    url = data[data['landmark_id'] == landMarkID]['url'].head(IMAGES_NUMBER)
    displayLandmarkImagesLarge(url, "")

pd.Series(data['landmark_id'].value_counts().keys())[1:4]

**Delete folders**

In [0]:
def deleteFolder(directory): 
  try:
      if os.path.exists(directory):
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print('Failed to delete %s. Reason: %s' % (file_path, e))
        
        os.removedirs(directory)
  except  Exception as e:
      print('Failed to delete %s. Reason: %s' % (file_path, e))

deleteFolder('/content/Data') 

**Create folders**

In [0]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
createFolder('/content/Data')        
createFolder('/content/Data/test_images_from_train')
createFolder('/content/Data/train_images_model')
createFolder('/content/Data/validation_images_model')  
createFolder('/content/Data/test_images')  
createFolder('/content/Data/train_images_model_resize')   

**Create lists for test, train and validation set**

In [0]:
TARGET_SIZE = 128 #imports images of resolution 128x128

#change URLs to resize images to target size
def overwrite_urls(df):
    def reso_overwrite(url_tail, reso=TARGET_SIZE):
        pattern = 's[0-9]+'
        search_result = re.match(pattern, url_tail)
        if search_result is None:
            return url_tail
        else:
            return 's{}'.format(reso)
    
    def join_url(parsed_url, s_reso):
        parsed_url[-2] = s_reso
        return '/'.join(parsed_url)
    
    df = df[df.url.apply(lambda x: len(x.split('/'))>1)]
    parsed_url = df.url.apply(lambda x: x.split('/'))
    train_url_tail = parsed_url.apply(lambda x: x[-2])
    resos = train_url_tail.apply(lambda x: reso_overwrite(x, reso=TARGET_SIZE))

    overwritten_df = pd.concat([parsed_url, resos], axis=1)
    overwritten_df.columns = ['url', 's_reso']
    df['url'] = overwritten_df.apply(lambda x: join_url(x['url'], x['s_reso']), axis=1)
    return df

data_sample_resize = overwrite_urls(data_sample)
print ('1. URLs overwritten')


'''Split to test and train'''
data_test = pd.DataFrame(columns = ['id','url','landmark_id'])
data_training_all = pd.DataFrame(columns = ['id','url','landmark_id'])
#percent_test = 0.01 #takes 1% from each class as holdout data


random.seed(42)
for landmark_id in set(data_sample_resize['landmark_id']):
    #directory = os.path.join(basepath, 'landmark_id') 
    #createFolder(directory)
    #os.chdir(directory) 
 
    n = 1
    t = data_sample_resize[(data_sample_resize.landmark_id == landmark_id)] #get all images for a landmark id
    
    if(len(t.id) <= 10):
      percent_test = 0.25
    else:
      if(len(t.id) > 10 and len(t.id) <= 100):
        percent_test = 0.05
      else:
        percent_test = 0.01
    
    i = 0
    r =[]
    while i < len(t.id):
        it = i
        try:
          ret = urlopen(t.url.iloc[it])
          r.append(t.id.iloc[it])  #create a list of all these images
          i += 1
        except HTTPError:
          i += 1
        
    test = random.sample(r, int(percent_test*len(r))) #randomly pick a sample of 1% images from list 'r'
    training = list(set(r) - set(test))  #get the remaining images
    data_t = data_sample_resize[data_sample_resize.id.isin(test)] #holdout dataset
    data_tr = data_sample_resize[data_sample_resize.id.isin(training)] #training dataset
    data_test = data_test.append(data_t)  
    data_training_all = data_training_all.append(data_tr)
    n+=1

print ('2. train and test set created')


'''Split into train and validation set'''
data_valid = pd.DataFrame(columns = ['id','url','landmark_id'])
data_train = pd.DataFrame(columns = ['id','url','landmark_id'])
percent_validation = 0.2 #takes 20% from each class as holdout data

random.seed(42)
for landmark_id in set(data_training_all['landmark_id']):
    n=1
    t = data_training_all[(data_training_all.landmark_id == landmark_id)]
    i = 0
    r =[]
    while i < len(t.id):
        it = i
        r.append(t.id.iloc[it])
        i += 1
        
    valid = random.sample(r,int(percent_validation*len(r)))
    train = list(set(r) - set(valid)) 
    data_v = data_training_all[data_training_all.id.isin(valid)]
    data_t = data_training_all[data_training_all.id.isin(train)]
    data_valid = data_valid.append(data_v)
    data_train = data_train.append(data_t)
    n+=1

print ('3. train and validation set created')

data_test = data_test.sort_values(by = ['landmark_id', 'id'])
print(data_test)
subfile = '/content/Data/test.csv'
with open(subfile, 'w') as csvfile:
  csvfile.write('id,url\n')
  i = 0
  for id in data_test['id']:
    csvfile.write('{0},{1}\n'.format(id,data_test['url'].iloc[i] ))
    i += 1
print('test.csv created')    
print (len(data_train))
print (len(data_valid))
print (len(data_test))

**Check data distribution**

In [0]:
# Check data distribution
%matplotlib inline


colors = np.array(['#4285f4','#34a853','#fbbc05','#ea4335'])
#Define the order in which to display the graph
order = ['1-5','5-10','10-50','50-100','100-200','200-500','>=500']
f, (ax1, ax2,ax3) = plt.subplots(1, 3,figsize=(15,5))


def plot_distribution(data_f, data_k, axis):
    #data['landmark_id'].value_counts()
    x=data_f.landmark_id.value_counts().index
    y=pd.DataFrame(data_f.landmark_id.value_counts())

    #Create a variable to group the number of image sin each class
    y['Number of images'] = np.where(y['landmark_id']>=500,'>=500',y['landmark_id'])
    y['Number of images'] = np.where((y['landmark_id']>=200) & (y['landmark_id']<500),'200-500',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=100) & (y['landmark_id']<200),'100-200',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=50) & (y['landmark_id']<100),'50-100',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=10) & (y['landmark_id']<50),'10-50',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=5) & (y['landmark_id']<10),'5-10',y['Number of images'])
    y['Number of images'] = np.where((y['landmark_id']>=0) & (y['landmark_id']<5),'1-5',y['Number of images'])

    y['Number of images'].value_counts().loc[order].plot(kind = 'bar',color = colors,width = 0.8, ax=axis)
    axis.set_xlabel('Number of images')
    axis.set_ylabel('Number of classes')
    axis.set_title(data_k)
    
plot_distribution(data_train, 'Training', ax1)
plot_distribution(data_valid, 'Validation', ax2)
plot_distribution(data_sample, 'Filtered Classes', ax3)


**Fetch images in folders**

In [0]:
#Remember to create the three folders for train-validation-test inside your Data directoy in order to be able to execute this code!
def fetch_image(path,folder):
    url = path
    response=requests.get(url, stream = True)
    with open('/content/Data/' + folder + '/image.jpg', 'wb') as out_file:
        shutil.copyfileobj(response.raw, out_file)
    del response
    
    
'''TRAIN SET - fetch images for the resized URLs and save in the already created directory train_images_model'''
i=0
for link in data_train['url']:              #looping over links to get images
    if os.path.exists('/content/Data/train_images_model/'+str(data_train['id'].iloc[i])+'.jpg'):
        i+=1
        continue
    fetch_image(link, 'train_images_model')
    os.rename('/content/Data/train_images_model/image.jpg', '/content/Data/train_images_model/' + str(data_train['id'].iloc[i]) + '.jpg')
    i+=1
#    if(i==50):   #uncomment to test in your machine
#        break
print('4. Train images fetched')

    
i=0
for link in data_valid['url']:              #looping over links to get images
    if os.path.exists('/content/Data/validation_images_model/' + str(data_valid['id'].iloc[i]) + '.jpg'):
        i += 1 
        continue
    fetch_image(link, 'validation_images_model')
    os.rename('/content/Data/validation_images_model/image.jpg', '/content/Data/validation_images_model/' + str(data_valid['id'].iloc[i]) + '.jpg')
    i += 1
#    if(i==50):   #uncomment to test in your machine
#        break
print('5. Validation images fetched')

i = 0
for link in data_test['url']:              #looping over links to get images
    if os.path.exists('/content/Data/test_images_from_train/' + str(data_test['id'].iloc[i]) + '.jpg'):
        i += 1
        continue
    fetch_image(link, 'test_images_from_train')
    os.rename('/content/Data/test_images_from_train/image.jpg', '/content/Data/test_images_from_train/' + str(data_test['id'].iloc[i]) + '.jpg')
    i += 1
#    if(i==50):   #uncomment to test in your machine
#        break
print('6. Test images from train fetched')

i=0
for link in data_test['url']:              #looping over links to get images
    #print(data_test['id'].iloc[i])
    if os.path.exists('/content/Data/test_images/' + str(data_test['id'].iloc[i]) + '.jpg'):
        i += 1
        continue
    fetch_image(link, 'test_images')
    os.rename('/content/Data/test_images/image.jpg', '/content/Data/test_images/' + str(data_test['id'].iloc[i]) + '.jpg')
    i += 1
print('7. Test images fetched')

**Visualization of images**

In [0]:
# Visualize 5 images for each of the first 4 landmarks, ordered by the number of occurences.

IDs = pd.Series(data_train['id'].value_counts().keys())[0:5]
for ID in IDs:
    url = data_train[data_train['id'] == ID]['url']
    displayLandmarkImagesLarge(url, "")

**Create subfolders for landmark_ID**

In [0]:
##create folders for landmark IDs in Training folder
train_data = data_train

temp = pd.DataFrame(data_train.landmark_id.value_counts())
temp.reset_index(inplace = True)
temp.columns = ['landmark_id', 'count']

def createfolders(dataset, folder):
    i = 0
    while i < len(dataset):
        landmark = str(dataset.landmark_id.iloc[i])
        path = '/content/Data/' + folder + '/' + landmark
        if not os.path.exists(path):
            os.makedirs(path)
        i += 1
createfolders(temp, 'train_images_model')
#make folders for landmark ID which had no images in training sets - required for codes running next
available = [int((x[0].split('/'))[-1]) for x in os.walk(r'/content/Data/train_images_model/') if len((x[0].split('/'))[-1]) > 0]
new = [str(x) for x in range(1000,1300) if x not in available]
for i in new:
    path = '/content/Data/train_images_model/' + i
    if not os.path.exists(path):
        os.makedirs(path)
print ('Train folders created')

rootdirpics = r'/content/Data/train_images_model/'
rootdirfolders = r'/content/Data/train_images_model/'

def transformdata(data,path1, path2):

    n = 1
    for landmark_id in set(data['landmark_id']):
        t = data[(data.landmark_id == landmark_id)]
        i = 1
        r = []
        while i <= len(t.id):
            it = i - 1
            r.append(t.id.iloc[it])
            i += 1
        for files in os.listdir(rootdirpics):    # loop through startfolders
            inpath = path1 + files
            folder = str(landmark_id)
            outpath = path2 + folder  
            if ((files.split('.')[0] in r) & (os.path.getsize(inpath) >1000)):
#                 print('move')
                shutil.move(inpath, outpath)
            elif ((files.split('.')[0] in r) & (os.path.getsize(inpath) <= 1000)):
                os.remove(inpath)
        n += 1

transformdata(train_data,rootdirpics, rootdirfolders)
print ('Train images moved')

##create folders for landmark IDs in Validation folder

temp = pd.DataFrame(data_valid.landmark_id.value_counts())
temp.reset_index(inplace = True)
temp.columns = ['landmark_id', 'count']
createfolders(temp, 'validation_images_model')
print ('Validation folders created')

#make folders for landmark ID which had no images in validation sets - required for codes running next
available = [int((x[0].split('/'))[-1]) for x in os.walk(r'/content/Data/validation_images_model/') if len((x[0].split('/'))[-1]) > 0]
new = [str(x) for x in range(1000,1300) if x not in available]
for i in new:
    path = '/content/Data/validation_images_model/' + i
    if not os.path.exists(path):
        os.makedirs(path)

rootdirpics = r'/content/Data/validation_images_model/'
rootdirfolders = r'/content/Data/validation_images_model/'
transformdata(data_valid,rootdirpics, rootdirfolders)
print ('Validation images moved')

##create folders for landmark IDs in Test folder (required for creating dataset with ImageFolder)

temp = pd.DataFrame(data_test.landmark_id.value_counts())
temp.reset_index(inplace = True)
temp.columns = ['landmark_id','count']
createfolders(temp,'test_images_from_train')
print ('Test folders created')

#make folders for landmark ID which had no images in validation sets - required for codes running next
available = [int((x[0].split('/'))[-1]) for x in os.walk(r'/content/Data/test_images_from_train/') if len((x[0].split('/'))[-1]) > 0]
new = [str(x) for x in range(1000,1300) if x not in available]
for i in new:
    path = '/content/Data/test_images_from_train/' + i
    if not os.path.exists(path):
        os.makedirs(path)

rootdirpics = r'/content/Data/test_images_from_train/'
rootdirfolders = r'/content/Data/test_images_from_train/'
transformdata(data_test,rootdirpics, rootdirfolders)
print ('Test images moved')

**Set Arguments**

In [0]:
DEVICE = 'cuda' 

NUM_CLASSES = len(landmark_list)
print('num classes: {}'.format(NUM_CLASSES))
BATCH_SIZE = 128

LR = 1e-2       
MOMENTUM = 0.9      
WEIGHT_DECAY = 5e-5 

NUM_EPOCHS = 10      
STEP_SIZE = 8       
GAMMA = 0.1          

LOG_FREQUENCY = 10

NUM_IMAGES_PER_FILE = 20

**Prepare Dataset**

In [0]:
# Prepare Pytorch train/test Datasets

DATA_DIR_training = '/content/Data/train_images_model/'
DATA_DIR_validation = '/content/Data/validation_images_model/'
DATA_DIR_test= '/content/Data/test_images_from_train/'

train_dataset = torchvision.datasets.ImageFolder(DATA_DIR_training, transform=train_transform)
val_dataset = torchvision.datasets.ImageFolder(DATA_DIR_validation, transform=eval_transform)
test_dataset = torchvision.datasets.ImageFolder(DATA_DIR_test, transform=eval_transform)

# Check dataset sizes
print('Train Dataset: {}'.format(train_dataset.__len__()))#len(train_dataset)))
print('Test Dataset: {}'.format(test_dataset.__len__()))#len(test_dataset)))
print('Validation Dataset: {}'.format(val_dataset.__len__()))#len(val_dataset)))


**Check Empty folder**

In [0]:
count = -1
check = 0
for i in range(len(train_dataset)):

  if count != train_dataset.__getitem__(i)[1]:
    
    if check != train_dataset.__getitem__(i)[1]:
      print('check = %d - label = %d'%(check, (((train_dataset.__getitem__(i)[1])))))
      check += 1
    #print('%d' %i)
    count = train_dataset.__getitem__(i)[1]
    check += 1

**Prepare Dataloaders**

In [0]:
# Dataloaders iterate over pytorch datasets and transparently provide useful functions (e.g. parallelization and shuffling)
train_dataloader = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers = 4, drop_last = True)
val_dataloader = DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = False, num_workers = 4)
test_dataloader = DataLoader(test_dataset, batch_size = 25, shuffle = False, num_workers = 4)

**Prepare Network**

In [0]:
#define resnet50 pre-trained on ImageNet dataset
net = torch.hub.load('pytorch/vision:v0.4.2','resnet50', pretrained = True)
net.fc = nn.Linear(2048, NUM_CLASSES)

**Prepare Training**

In [0]:
criterion = nn.CrossEntropyLoss() 

parameters_to_optimize = net.parameters() 

optimizer = optim.SGD(parameters_to_optimize, lr = LR, momentum = MOMENTUM, weight_decay = WEIGHT_DECAY)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = STEP_SIZE, gamma = GAMMA)

**Accuracy Evaluation**

In [0]:
#This function is used for the computation of the accuracy where needed in the following sections
def test_accuracy(net, dataloader, type_of_set):  # net=model, dataloader=iterable obj
    correct = 0
    total = 0
    net.train(False)  #net.eval()  # important for deactivating dropout and correctly use batchnorm accumulated statistics
    with torch.no_grad(): #temporarily set all the requires_grad flag to false
        for data in dataloader:
            images, labels = data
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            outputs = net(images)  # predictions
            _, predicted = torch.max(outputs.data, 1)  # predicted labels
            total += labels.size(0)
            correct += torch.sum(predicted == labels.data).data.item()  # compare with ground truth
    accuracy = 100 * correct / total
    print('Accuracy of the network on the %s set: %d %%' %(type_of_set, accuracy))
    net.train(True)
    return accuracy

**Train**

In [0]:
net = net.to(DEVICE) 

cudnn.benchmark 
n_epoch = NUM_EPOCHS
losses = np.empty(n_epoch)
j = 0
current_step = 0
accuracies_train = np.empty(n_epoch)
accuracies_val =  np.empty(n_epoch)
n_loss_print = len(train_dataloader)
criterion = nn.CrossEntropyLoss() 

  # Start iterating over the epochs
for epoch in range(n_epoch):
  running_loss = 0.0
  print('Starting epoch {}/{}, LR = {}'.format(epoch+1, n_epoch, scheduler.get_lr()))
  # Iterate over the dataset
  for images, labels in train_dataloader:
    # Bring data over the device of choice
    images = images.to(DEVICE)
    labels = labels.to(DEVICE)

    net.train() 

    optimizer.zero_grad() 

    outputs = net(images)

    loss = criterion(outputs, labels)
    running_loss += loss.item()
    
    if current_step % LOG_FREQUENCY == 0:
      print('Step {}, Loss {}'.format(current_step, loss.item()))

    loss.backward()  
    optimizer.step() 

    current_step += 1
  
  
  losses[j] = running_loss / n_loss_print
  accuracies_train[j] = test_accuracy(net, train_dataloader, 'train')  # at each epoch

  #EVALUATION ON VALIDATION
  accuracies_val[j] = test_accuracy(net, val_dataloader, 'validation')  # at each epoch
  j += 1

  # Step the scheduler
  scheduler.step() 


#plot
Num_epochs= np.linspace(1,n_epoch,n_epoch)
plt.plot(Num_epochs, accuracies_train, 'r')
plt.plot(Num_epochs, accuracies_val,   'g')
plt.legend(loc='best', labels = ('Accuracy on Train', 'Accuracy on Validation') )
plt.xlabel('n epochs')
plt.title('Accuracy on train and validation set')
plt.grid()
plt.show() 
plt.plot(Num_epochs, losses,'b', label = 'Training Loss') 
plt.xlabel('n epochs')
plt.legend(loc='best')
plt.title('Loss on training set')
plt.grid()
plt.show()  

**Test**

In [0]:
net = net.to(DEVICE) 
net.train(False) # Set Network to evaluation mode
test_dataloader = DataLoader(test_dataset, batch_size = 1, shuffle = False, num_workers = 4)
running_corrects = 0
predictions = []
Prob = []
GT = []
bs = 1
for images, labels in tqdm(test_dataloader):
  images = images.to(DEVICE)
  labels = labels.to(DEVICE)

  for j in range(bs):
   GT.append(labels[j].data.item())

  # Forward Pass
  outputs = net(images)

  sm = torch.nn.Softmax()
  probabilities = sm(outputs)
  #print(probabilities)
  top = torch.topk(probabilities, 1)
  #print(top.values)
  for j in range(bs):
    Prob.append(top.values[j].data.item())

  # Get predictions
  _, preds = torch.max(outputs.data, 1)
  for j in range(bs):
    predictions.append(preds[j].data.item())

  # Update Corrects
  running_corrects += torch.sum(preds == labels.data).data.item()

# Calculate Accuracy
accuracy = running_corrects / float(len(test_dataset))

print('Test Accuracy: {}%'.format(accuracy * 100))
print(predictions)
print(GT)
print(Prob)

**Submit predictions**

In [0]:
subfile1 = '/content/Data/sub_spezifinal.csv'

test = pd.read_csv('/content/Data/test.csv')

def create_predictions_csv(predictions, test_set_list):
    print ('starting...')
    file_len = len(test_set_list)
    print('Number of Test images:', file_len)
    
    j = 0
    prediction_list=[]
    for j in range(len(predictions)):
      prediction_list.append(predictions[j] + 1000)
    #print(prediction_list)
    
    k = 0 #contatore della lunghezza di prediction_list 
    num_csv = len(prediction_list) / NUM_IMAGES_PER_FILE
    print('divisione iniziale: ', num_csv)
    num_csv = int(num_csv)
    if len(prediction_list) % NUM_IMAGES_PER_FILE > 0:
      num_csv += 1 
    print('numero di file da creare: ',num_csv)

    splitted_subfile = subfile1.split('.')[0] #/content/Data/sub_spezifinal
    for num in range(0, num_csv):
      csv_to_open = splitted_subfile + str(num + 1) + '.csv'  #/content/Data/sub_spezifinal1.csv, /content/Data/sub_spezifinal2.csv ...
      print(csv_to_open)
      with open(csv_to_open, 'w') as csvfile:
          csvfile.write('id,landmark,match,prob\n')
          for i in range(0, NUM_IMAGES_PER_FILE):
            if(k < len(prediction_list)):
              id = test['id'][k]
              landmark = prediction_list[k]
              prob = Prob[k]
              csvfile.write('{0},{1},N,{2}\n'.format(id, landmark, prob))
              k += 1

create_predictions_csv(predictions, data_test)

**DELF**

In [0]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
from scipy.spatial import cKDTree
from skimage.feature import plot_matches
from skimage.measure import ransac
from skimage.transform import AffineTransform
from six import BytesIO
import tensorflow as tf
import tensorflow_hub as hub
from six.moves.urllib.request import urlopen
from statistics import mean

import glob
import os
from itertools import accumulate
import pandas as pd
from pandas import Series,DataFrame

def DELF(input_file, output_file):
  np.random.seed(10)
  pred = pd.read_csv(input_file) #csv file containing predictions for test images from neural network
  subfile = output_file

  with open(subfile, 'w') as csvfile:
    csvfile.write('id,landmark,match,prob\n')
    for zoro in range(0, len(pred.index)):
        class_folder = str(pred['landmark'][zoro])  #class_folder =  '1070'
        test_image_id = str(pred['id'][zoro])       #test_image_id = '5f516881e61501c4'
        prob = pred['prob'][zoro]                   #prob = 0.2625 
        
        def resize_image(srcfile, destfile, new_width = 256, new_height = 256):
            pil_image = Image.open(srcfile)
            pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
            pil_image_rgb = pil_image.convert('RGB')
            pil_image_rgb.save(destfile, format = 'JPEG', quality = 90)
            return destfile
        def resize_images_folder(srcfolder, destfolder='/content/Data/train_images_model_resize/%s'%(class_folder), new_width = 256, new_height = 256):
            os.makedirs(destfolder,exist_ok=True)
            for srcfile in glob.iglob(os.path.join('/content/Data/train_images_model/%s'%(class_folder), '*.[Jj][Pp][Gg]')):
                src_basename = os.path.basename(srcfile)
                destfile = os.path.join(destfolder,src_basename)
                resize_image(srcfile, destfile, new_width, new_height)
            return destfolder
    ###########################################################################################################################
        def get_resized_db_image_paths(destfolder = '/content/Data/train_images_model_resize/%s'%(class_folder)):
            return sorted(list(glob.iglob(os.path.join(destfolder, '*.[Jj][Pp][Gg]'))))

            
        resize_images_folder('/content/Data/train_images_model/%s'%(class_folder))
        db_images = get_resized_db_image_paths() ## list of image paths

        tf.reset_default_graph()
        tf.logging.set_verbosity(tf.logging.FATAL)

        m = hub.Module('https://tfhub.dev/google/delf/1')

        # The module operates on a single image at a time, so define a placeholder to
        # feed an arbitrary image in.
        image_placeholder = tf.placeholder(
            tf.float32, shape = (None, None, 3), name = 'input_image')

        module_inputs = {
            'image': image_placeholder,
            'score_threshold': 100.0,
            'image_scales': [0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0],
            'max_feature_num': 1000,
        }

        module_outputs = m(module_inputs, as_dict=True)

        def image_input_fn(image_files):
          filename_queue = tf.train.string_input_producer(
              image_files, shuffle = False)
          reader = tf.WholeFileReader()
          _, value = reader.read(filename_queue)
          image_tf = tf.image.decode_jpeg(value, channels=3)
          return tf.image.convert_image_dtype(image_tf, tf.float32)

        try:
          image_tf = image_input_fn(db_images) ##  training images path list inputted
        except ValueError:
          continue
          
        with tf.train.MonitoredSession() as sess:
            results_dict = {}  # Stores the locations and their descriptors for each image
            for image_path in db_images:
                image = sess.run(image_tf)
                #print('Extracting locations and descriptors from %s' % image_path)
                results_dict[image_path] = sess.run(
                    [module_outputs['locations'], module_outputs['descriptors']],
                    feed_dict={image_placeholder: image})
      #########################################################################################################################          
        def compute_locations_and_descriptors(image_path):
            tf.reset_default_graph()
            tf.logging.set_verbosity(tf.logging.FATAL)

            m = hub.Module('https://tfhub.dev/google/delf/1')

            # The module operates on a single image at a time, so define a placeholder to
            # feed an arbitrary image in.
            image_placeholder = tf.placeholder(
                tf.float32, shape = (None, None, 3), name = 'input_image')

            module_inputs = {
                'image': image_placeholder,
                'score_threshold': 100.0,
                'image_scales': [0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0],
                'max_feature_num': 1000,
            }

            module_outputs = m(module_inputs, as_dict=True)

            def image_input_fn(image_files):
              filename_queue = tf.train.string_input_producer(
                  image_files, shuffle=False)
              reader = tf.WholeFileReader()
              _, value = reader.read(filename_queue)
              image_tf = tf.image.decode_jpeg(value, channels=3)
              return tf.image.convert_image_dtype(image_tf, tf.float32)

            image_tf = image_input_fn([image_path])

            with tf.train.MonitoredSession() as sess:
                image = sess.run(image_tf)
                #print('Extracting locations and descriptors from %s' % image_path)
                return sess.run(
                    [module_outputs['locations'], module_outputs['descriptors']],
                    feed_dict={image_placeholder: image})
            
    ###############################################################################################################################
        locations_agg = np.concatenate([results_dict[img][0] for img in db_images])
        descriptors_agg = np.concatenate([results_dict[img][1] for img in db_images])
        for desc in locations_agg:
              num_features = locations_agg.shape[0]
              #print("DB image has %d features" % num_features)
        accumulated_indexes_boundaries = list(accumulate([results_dict[img][0].shape[0] for img in db_images]))
        
    ###############################################################################################################################
        d_tree = cKDTree(descriptors_agg) # build the KD tree
        
    ###############################################################################################################################
        query_image = '/content/Data/test_images/%s.jpg'%(test_image_id)
        #query_image= '/content/Data/test_images/5f516881e61501c4.jpg'
        #query_image= '/content/Data/test_images/5f516881e61501c4.jpg'
        def preprocess_query_image(imagepath):
            '''
            Resize the query image and return the resized image path.
            '''
            query_temp_folder_name = 'query_temp_folder'
            query_temp_folder = os.path.join(os.path.dirname(imagepath), query_temp_folder_name)
            os.makedirs(query_temp_folder, exist_ok = True)
            query_basename = os.path.basename(imagepath)
            destfile = os.path.join(query_temp_folder, query_basename)
            resized_image = resize_image(imagepath, destfile)
            return resized_image

        resized_image = preprocess_query_image(query_image)
        
    ###############################################################################################################################
        query_image_locations, query_image_descriptors = compute_locations_and_descriptors(resized_image)
        num_features_query = query_image_locations.shape[0]
        print("Query image has %d features" % num_features_query)
        
    ###############################################################################################################################
        distance_threshold = 0.8
        # K nearest neighbors
        K = 10
        distances, indices = d_tree.query(
            query_image_descriptors, distance_upper_bound = distance_threshold, k = K, n_jobs = -1)

        # Find the list of unique accumulated/aggregated indexes
        unique_indices = np.array(list(set(indices.flatten())))

        unique_indices.sort()
        if unique_indices[-1] == descriptors_agg.shape[0]:
            unique_indices = unique_indices[:-1]
            
    ###############################################################################################################################
        unique_image_indexes = np.array(
            list(set([np.argmax([np.array(accumulated_indexes_boundaries) > index]) 
                      for index in unique_indices])))
        unique_image_indexes
        
    ###############################################################################################################################
        def image_index_2_accumulated_indexes(index, accumulated_indexes_boundaries):
            '''
            Image index to accumulated/aggregated locations/descriptors pair indexes.
            '''
            if index > len(accumulated_indexes_boundaries) - 1:
                return None
            accumulated_index_start = None
            accumulated_index_end = None
            if index == 0:
                accumulated_index_start = 0
                accumulated_index_end = accumulated_indexes_boundaries[index]
            else:
                accumulated_index_start = accumulated_indexes_boundaries[index-1]
                accumulated_index_end = accumulated_indexes_boundaries[index]
            return np.arange(accumulated_index_start,accumulated_index_end)
        
    ###############################################################################################################################
        def get_locations_2_use(image_db_index, k_nearest_indices, accumulated_indexes_boundaries):
            '''
            Get a pair of locations to use, the query image to the database image with given index.
            Return: a tuple of 2 numpy arrays, the locations pair.
            '''
            image_accumulated_indexes = image_index_2_accumulated_indexes(image_db_index, accumulated_indexes_boundaries)
            locations_2_use_query = []
            locations_2_use_db = []
            for i, row in enumerate(k_nearest_indices):
                for acc_index in row:
                    if acc_index in image_accumulated_indexes:
                        locations_2_use_query.append(query_image_locations[i])
                        locations_2_use_db.append(locations_agg[acc_index])
                        break
            return np.array(locations_2_use_query), np.array(locations_2_use_db)
        
    ###############################################################################################################################
        
        # Array to keep track of all candidates in database.
        inliers_counts = []
        # Read the resized query image for plotting.
        img_1 = mpimg.imread(resized_image)
        comparison = 0
        top3 = []
        flag = 0
        for index in unique_image_indexes:
          
          if comparison > 30:
            break
            
          locations_2_use_query, locations_2_use_db = get_locations_2_use(index, indices, accumulated_indexes_boundaries)
          # Perform geometric verification using RANSAC.
          try:
            _, inliers = ransac(
                (locations_2_use_db, locations_2_use_query), # source and destination coordinates
                AffineTransform,
                min_samples = 3,
                residual_threshold = 20,
                max_trials = 1000)
          except ValueError:
            #print('no sample')
            inliers = None
            
          # If no inlier is found for a database candidate image, we continue on to the next one.
          if inliers is None or len(inliers) == 0:
              continue
          # the number of inliers as the score for retrieved images.
          inliers_counts.append({"index": index, "inliers": sum(inliers)})
          print('Found inliers for image {} -> {}'.format(index, sum(inliers)))
          # Visualize correspondences.
          _, ax = plt.subplots()
          img_2 = mpimg.imread(db_images[index])
          inlier_idxs = np.nonzero(inliers)[0]
          plot_matches(
              ax,
              img_1,
              img_2,
              locations_2_use_db,
              locations_2_use_query,
              np.column_stack((inlier_idxs, inlier_idxs)),
              matches_color = 'b')
          ax.axis('off')
          ax.set_title('DELF correspondences (class = {})'.format(class_folder))
          plt.show()
          
      ###############################################################################################################################
          inliers_list = []
          for inl in inliers_counts:
              inliers_list.append(inl['inliers'])

          if len(inliers_list) > 0:
              mean_features = mean(inliers_list)
          else:
              mean_features = 0
          print(inliers_counts)
          print(inliers_list)
          print('Average number of inliers: {}'.format(mean(inliers_list)))
          comparison += 1
          if sum(inliers) >= 25:
            csvfile.write('{0},{1},1,{2}\n'.format(test_image_id, class_folder, prob))
            
            flag = 1
            break


        #deciding landmark or not based on threshold
        if flag == 0:
          inliers_list = sorted(inliers_list, reverse = True)
          top3 = inliers_list[:3]
          mean_max_inliers = mean(top3)
          print('Mean value of top3 number of inliers found: {}'.format(mean_max_inliers))

          if mean_max_inliers < 5: 
            csvfile.write('{0},{1},N,{2}\n'.format(test_image_id, class_folder, prob))
          else:
            if mean_max_inliers > 8:
              csvfile.write('{0},{1},1,{2}\n'.format(test_image_id, class_folder, prob))
            else:
              csvfile.write('{0},{1},0,{2}\n'.format(test_image_id, class_folder, prob))
        
        flag = 0            

In [0]:
#run this cell for every csv file created, changeing every time the input and output file
#1--> '/content/Data/sub_spezifinal1.csv', '/content/Data/sub_spezifinal_delf1.csv'
#2--> '/content/Data/sub_spezifinal2.csv', '/content/Data/sub_spezifinal_delf2.csv'
#3--> '/content/Data/sub_spezifinal3.csv', '/content/Data/sub_spezifinal_delf3.csv'
#4--> '/content/Data/sub_spezifinal4.csv', '/content/Data/sub_spezifinal_delf4.csv'
#5--> '/content/Data/sub_spezifinal5.csv', '/content/Data/sub_spezifinal_delf5.csv'
num_csv = 14 #every time add the right number 
subfile_input = '/content/Data/sub_spezifinal'
subfile_output = '/content/Data/sub_spezifinal_delf'

for i in range(0, num_csv):
  input_file = subfile_input + str(i + 1) + '.csv'
  output_file = subfile_output + str(i + 1) + '.csv'
  DELF(input_file, output_file)

**Merge of DELF file**

In [0]:
delf_final = '/content/Data/sub_spezifinal_delf_final.csv'
subfile = '/content/Data/sub_spezifinal_delf' 

#delf_final = '/content/Data/sub_spezifinal_final.csv'
#subfile = '/content/Data/sub_spezifinal'
num_csv = 14 #every time add the right number 

with open(delf_final, 'w') as csvfile:
  csvfile.write('id,landmark,match,prob\n')
  for i in range(0, num_csv):
    csv_to_open = subfile + str(i + 1) + '.csv'
    pred = pd.read_csv(csv_to_open)
    for j in range(len(pred)):
      csvfile.write('{0},{1},{2},{3}\n'.format(pred['id'][j], pred['landmark'][j], pred['match'][j], pred['prob'][j]))

**Computation of GAP**

In [0]:
delf_results = pd.read_csv('/content/Data/sub_spezifinal_delf_final.csv')
print(delf_results)
N = len(delf_results)
print('N = ', N)
M = 0
true_positive = 0
false_positive = 0
results = []
gap_values = []
for i in range(0, N):
  match = delf_results['match'][i]
  #print(match)
  if(str(match) != 'N'):
    M += 1
    results.append([delf_results['id'][i], delf_results['landmark'][i], delf_results['match'][i], delf_results['prob'][i]])
print('M = ', M)    

results = sorted(results, key = lambda results: results[3], reverse = True)
print(results)
GAP = 0

for i in range(M):
  if(results[i][2] == '1'):
    #print('caso == 1')
    true_positive += 1
    rel = 1
    print('rel({}) = {}'.format(i, rel))
  else:
    #print(results[i][2])
    if(results[i][2] == '0'):
      #print('caso == 0')
      false_positive += 1  
      rel = 0
      print('rel({}) = {}'.format(i, rel))

  #if(true_positive != 0 and false_positive != 0):
  P = true_positive / (true_positive + false_positive)
  print('P({}) = {}'.format(i, P))
  GAP += P*rel
  gap_values.append(GAP/M)
  print('GAP numerator: ', GAP)

plt.plot(range(M), gap_values, 'r')
#plt.legend(loc='best', labels = ('Accuracy on Train', 'Accuracy on Validation') )
plt.xlabel('number of predictions')
plt.title('GAP on test set')
plt.grid()
plt.show() 

GAP = (GAP/M)*100
print('accuracy GAP: {}%'.format(GAP))

**DELF on two sample Images** 

In [0]:
# IMAGE_1_URL = 'https://images.unsplash.com/photo-1522184808165-a39a75502037?ixlib=rb-0.3.5&ixid=eyJhcHBfaWQiOjEyMDd9&s=9058a9f269234b4d7b6dc7bd7066d7e8&auto=format&fit=crop&w=1350&q=80'
# IMAGE_2_URL = 'https://images.unsplash.com/photo-1479660656269-197ebb83b540?ixlib=rb-0.3.5&ixid=eyJhcHBfaWQiOjEyMDd9&s=88b85498e4c076fce0ed0c086543afb6&auto=format&fit=crop&w=1952&q=80'

IMAGE_1_URL = '/content/Data/test_images_from_train/1140/814b5c99a1218cb3.jpg'
IMAGE_2_URL = '/content/Data/test_images_from_train/1140/34347fabe6a816f8.jpg'


# The names that will be used for the resized local images.

IMAGE_1_JPG = 'image_1.jpg'
IMAGE_2_JPG = 'image_2.jpg'

#@title The images that will be processed by DELF
def download_and_resize_image(url, filename, new_width=128, new_height=128):
    if 'http' in url:
        response = urlopen(url)
        image_data = response.read()
        pil_image = Image.open(BytesIO(image_data))
    else:
        pil_image = Image.open(url)
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    pil_image_rgb = pil_image.convert('RGB')
    pil_image_rgb.save(filename, format='JPEG', quality=90)

download_and_resize_image(IMAGE_1_URL, IMAGE_1_JPG)
download_and_resize_image(IMAGE_2_URL, IMAGE_2_JPG)

def show_images(image_path_list):
    plt.figure()
    for i, image_path in enumerate(image_path_list):
        plt.subplot(1, len(image_path_list), i+1)
        plt.imshow(np.asarray(Image.open(image_path)))
        plt.title(image_path)
        plt.grid(False)
        plt.yticks([])
        plt.xticks([])
    plt.show()

show_images([IMAGE_1_JPG, IMAGE_2_JPG])

def image_input_fn(image_files):
    filename_queue = tf.train.string_input_producer(
        image_files, shuffle=False)
    reader = tf.WholeFileReader()
    _, value = reader.read(filename_queue)
    image_tf = tf.image.decode_jpeg(value, channels=3)
    return tf.image.convert_image_dtype(image_tf, tf.float32)


tf.reset_default_graph()
tf.logging.set_verbosity(tf.logging.FATAL)

m = hub.Module('https://tfhub.dev/google/delf/1')

# The module operates on a single image at a time, so define a placeholder to
# feed an arbitrary image in.
image_placeholder = tf.placeholder(
    tf.float32, shape=(None, None, 3), name='input_image')

module_inputs = {
    'image': image_placeholder,
    'score_threshold': 100.0,
    'image_scales': [0.25, 0.3536, 0.5, 0.7071, 1.0, 1.4142, 2.0],
    'max_feature_num': 1000,
}

module_outputs = m(module_inputs, as_dict=True)

image_tf = image_input_fn([IMAGE_1_JPG, IMAGE_2_JPG])

with tf.train.MonitoredSession() as sess:
    results_dict = {}  # Stores the locations and their descriptors for each image
    for image_path in [IMAGE_1_JPG, IMAGE_2_JPG]:
        image = sess.run(image_tf)
        print('Extracting locations and descriptors from %s' % image_path)
        results_dict[image_path] = sess.run(
            [module_outputs['locations'], module_outputs['descriptors']],
            feed_dict={image_placeholder: image})


#@title TensorFlow is not needed for this post-processing and visualization
def match_images(results_dict, image_1_path, image_2_path):
    distance_threshold = 0.8

    # Read features.
    locations_1, descriptors_1 = results_dict[image_1_path]
    num_features_1 = locations_1.shape[0]
    print("Loaded image 1's %d features" % num_features_1)
    locations_2, descriptors_2 = results_dict[image_2_path]
    num_features_2 = locations_2.shape[0]
    print("Loaded image 2's %d features" % num_features_2)

    # Find nearest-neighbor matches using a KD tree.
    d1_tree = cKDTree(descriptors_1)
    _, indices = d1_tree.query(
        descriptors_2, distance_upper_bound=distance_threshold)

  # Select feature locations for putative matches.
    locations_2_to_use = np.array([
        locations_2[i,]
        for i in range(num_features_2)
        if indices[i] != num_features_1
    ])
    locations_1_to_use = np.array([
        locations_1[indices[i],]
        for i in range(num_features_2)
        if indices[i] != num_features_1
    ])

  # Perform geometric verification using RANSAC.
    _, inliers = ransac(
        (locations_1_to_use, locations_2_to_use),
        AffineTransform,
        min_samples=3,
        residual_threshold=20,
        max_trials=1000)
    # the number of inliers as the score for retrieved images.
    print('Found %d inliers' % sum(inliers))

    # Visualize correspondences.
    _, ax = plt.subplots()
    img_1 = mpimg.imread(image_1_path)
    img_2 = mpimg.imread(image_2_path)
    inlier_idxs = np.nonzero(inliers)[0]
    plot_matches(
        ax,
        img_1,
        img_2,
        locations_1_to_use,
        locations_2_to_use,
        np.column_stack((inlier_idxs, inlier_idxs)),
        matches_color='b')
    ax.axis('off')
    ax.set_title('DELF correspondences')
    plt.figure(figsize=(18,24))
    plt.show()

match_images(results_dict, IMAGE_1_JPG, IMAGE_2_JPG)

